In [1]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QToolTip, QFileDialog,QButtonGroup,
                             QLabel, QRadioButton, QComboBox, QLineEdit, QPushButton, QGridLayout)
from PyQt5.QtGui import QIcon
from PyQt5.QtGui import QFont
from playsound import playsound
from scipy import stats
import scipy.io
from scipy.io import wavfile
import sounddevice as sd
import numpy as np
import pandas as pd
import pygame
import time
from numpy.lib import stride_tricks
from PyQt5 import QtGui
import python_speech_features
import librosa
import csv
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvus
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from sklearn.externals import joblib
import os
import glob,os.path
'''
#QtCore module contains the core non-GUI functionality. This module is used for working with time, files and directories, various data types, streams, URLs, mime types, threads or processes
#QtGui contains classes for windowing system integration, event handling, 2D graphics, basic imaging, fonts and text
#QtWidgets module contains classes that provide a set of UI elements to create classic desktop-style user interfaces
#QtNetwork module contains the classes for network programming.
#QtWebSockets module contains classes that implement the WebSocket protocol.
#QtSql module provides classes for working with databases.
#QtXml contains classes for working with XML files
#QtMultimedia contains classes to handle multimedia content and APIs to access camera and radio functionality.
'''

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


'\n#QtCore module contains the core non-GUI functionality. This module is used for working with time, files and directories, various data types, streams, URLs, mime types, threads or processes\n#QtGui contains classes for windowing system integration, event handling, 2D graphics, basic imaging, fonts and text\n#QtWidgets module contains classes that provide a set of UI elements to create classic desktop-style user interfaces\n#QtNetwork module contains the classes for network programming.\n#QtWebSockets module contains classes that implement the WebSocket protocol.\n#QtSql module provides classes for working with databases.\n#QtXml contains classes for working with XML files\n#QtMultimedia contains classes to handle multimedia content and APIs to access camera and radio functionality.\n'

In [ ]:
class Window(QWidget): #inherits from the QWidget class. QWidget widget is the base class of all user interface objects in PyQt5.

        def __init__(self):
                super().__init__() #super() method returns the parent object of the Window class and we call its constructor. 
                self.initUI()
        def initUI(self):

                #Naming The Widgets
                audioType = QLabel('Audio Input (.wav file): ')
                #audioType.setFont(QFont.setBold(self,True))
                fs = QLabel('Sampling Freq.(Hz)')
                time = QLabel('Duration(Sec)')
                predictionPart = QLabel('Prediction Part-50% Overlap')
                myFont = QtGui.QFont()
                myFont.setBold(True)
                predictionPart.setFont(myFont)
                modelSelect = QLabel('Select The Model')
                frameSize = QLabel('Frame Size (in ms)')
                windowLenEdit = QLabel('Window Length(ms)')
                predicitonResult = QLabel('Total Prediction')
                self.lbl = QLabel(self)
                self.Index = 0
                self.modelIndex = 0
                pygame.init()
                
                #Implementing Those Widgets
                self.nameOfAudio = QLineEdit()
                self.uploadAudio = QPushButton("Upload Audio")
                self.fsEdit = QComboBox() 
                ##### Reording Button #####
                
                self.timeDuration = QLineEdit()
                self.loadAudio = QPushButton("Load Data")
                self.loadAudio.setEnabled(False)
                self.plotEdit = QPushButton("Plot Data")
                self.plotEdit.setEnabled(False)
                self.playFrame = QPushButton('Play Frame')
                self.playFrame.setEnabled(False)
                self.figure = plt.figure(figsize=(5,5),dpi=120)
                self.canvas = FigureCanvus(self.figure)
                self.toolbar = NavigationToolbar(self.canvas,self)
                self.importModel = QPushButton("Import Model")
                self.importModel.setEnabled(False)
                self.processStart = QPushButton("Process")
                self.processStart.setEnabled(False)
                self.predictStart = QPushButton("Predict")
                self.predictStart.setEnabled(False)
                ##### Model Selection #####
                self.modelSelect = QComboBox()
                ##### Frame Size Selection #####
                self.frameSizeEdit = QComboBox()
                ##### Window Length Selection for Prediction #####
                self.windowLenEdit = QComboBox()
                self.modelGraph = QComboBox()
                self.Show = QPushButton("Show")
                self.Show.setEnabled(False)
                self.predictionRecord = QPushButton("Result")
                self.predictionRecord.setEnabled(False)
                self.totalPredictionResult = QLineEdit()
                self.cancelEdit = QPushButton("CANCEL")
                self.cancelEdit.setEnabled(False)
                self.back = QPushButton("<<")
                self.back.setEnabled(False)
                self.front = QPushButton(">>")
                self.front.setEnabled(False)
                self.showFrame = QLineEdit()
                self.startTime = QLineEdit()
                self.endTime = QLineEdit()
                self.reset = QPushButton('Reset')
                self.reset.setEnabled(False)
                
                
                #Filling In Details
                
                self.fsEdit.addItem('16000')
                self.fsEdit.addItem('44100')
               
                self.frameSizeEdit.addItem('100')
                self.frameSizeEdit.addItem('250')
                self.frameSizeEdit.addItem('500')
                #self.frameSizeEdit.addItem('500')
                #self.modelSelect.addItem('SVM')
                self.modelSelect.addItem('FNN')
                self.modelSelect.addItem('SVM')
                self.modelSelect.addItem('1D CNN')
                self.modelSelect.addItem('2D CNN')
                #self.modelSelect.addItem('1D CNN Without MFCC')
                self.windowLenEdit.addItem('200')
                self.windowLenEdit.addItem('500')
                self.windowLenEdit.addItem('1000')
                self.windowLenEdit.addItem('2000')
                self.windowLenEdit.addItem('5000')
                #self.modelGraph.addItem('Model Plot')
                self.modelGraph.addItem('K-fold Accuracy')
                self.modelGraph.addItem('K-fold Loss')
                self.modelGraph.addItem('Confusion-Matrix')
                

                #Setting Layout
                grid = QGridLayout()
                grid.setSpacing(5)              
                #1st Row
                #grid.addWidget(audioType, 1, 0, 1, 1)
                grid.addWidget(self.nameOfAudio,1,1,1,2)
                grid.addWidget(self.uploadAudio,1,0,1,1)
                grid.addWidget(fs , 1, 3, 1, 1)
                grid.addWidget(self.fsEdit,1,4,1,1)
                
                #2nd Row
                grid.addWidget(self.loadAudio,2,1,1,1)
                grid.addWidget(self.plotEdit, 2, 2, 1, 1)
                grid.addWidget(time, 2, 3, 1, 1)
                grid.addWidget(self.timeDuration, 2, 4, 1, 1)
                
                #3rd Row
                grid.addWidget(self.playFrame,3,0,1,1)
                grid.addWidget(self.toolbar, 3, 1, 1, 4)
                #4th Row
                grid.addWidget(self.canvas, 4, 0, 4, 4)
                grid.addWidget(self.lbl, 4,4,1,2)
                
                #5th Row
                grid.addWidget(predictionPart, 8, 2, 1, 1)
                
                #6th Row
                grid.addWidget(modelSelect, 9, 0, 1, 1)
                grid.addWidget(self.modelSelect, 9, 1, 1, 1)
                grid.addWidget(frameSize , 9, 2, 1, 1)
                grid.addWidget(self.frameSizeEdit, 9, 3, 1, 1)
                grid.addWidget(self.modelGraph, 9, 4, 1, 1)
                grid.addWidget(self.Show, 9, 5, 1, 1)
                
                #7th Row
                grid.addWidget(windowLenEdit , 10, 0, 1, 1)
                grid.addWidget(self.windowLenEdit, 10, 1, 1, 1)
                grid.addWidget(self.importModel, 10, 2, 1, 1)
                grid.addWidget(self.processStart,10, 3, 1, 1)
                grid.addWidget(self.predictStart,10, 4, 1, 1)
                #8th Row
                grid.addWidget(predicitonResult, 11, 0, 1, 1)
                grid.addWidget(self.totalPredictionResult, 11,1, 1, 3)
                self.totalPredictionResult.resize(220,80)
                grid.addWidget(self.predictionRecord, 11, 4, 1, 1)
                
                #9th Row
                grid.addWidget(self.back,12,0,1,1)
                grid.addWidget(self.startTime,12,1,1,1)
                grid.addWidget(self.showFrame,12,2,1,1)
                grid.addWidget(self.endTime,12,3,1,1)
                grid.addWidget(self.front,12,4,1,1)
                
                #10th row
                grid.addWidget(self.reset,13,4,1,1)
                grid.addWidget(self.cancelEdit,13,5,1,1)
                
                
                self.setLayout(grid)
                self.uploadAudio.clicked.connect(self.showDialog)
                self.loadAudio.clicked.connect(self.load)
                self.plotEdit.clicked.connect(self.plot)
                self.playFrame.clicked.connect(self.playframe)
                self.importModel.clicked.connect(self.importmodel)
                self.processStart.clicked.connect(self.process)
                self.predictStart.clicked.connect(self.predict)
                self.predictionRecord.clicked.connect(self.record)
                self.Show.clicked.connect(self.modelShow)
                self.back.clicked.connect(self.left)
                self.front.clicked.connect(self.right)
                self.reset.clicked.connect(self.Reset)
                self.cancelEdit.clicked.connect(self.cancel)
                
                                
                self.setGeometry(300, 300, 500, 400) #locates the window on the screen and sets it size(x,y,x+w,y+d)
                self.setWindowTitle('GUI for Audio Scene Prediction')
                #self.show(QIcon('FileName.png'))
                self.show()
        
        def str2int_fs(self):
            b = [int(y) for y in self.fsEdit.currentText()]
            c = 0;
            for i in b:
                c = c*10 + i
            return c
            
        def str2int_framesize(self):
            b = [int(y) for y in self.frameSizeEdit.currentText()]
            c = 0;
            for i in b:
                c = c*10 + i
            return c
        
        def str2int_winlen(self):
            b = [int(y) for y in self.windowLenEdit.currentText()]
            c = 0;
            for i in b:
                c = c*10 + i
            return c
        
        def showDialog(self):
                self.fname = QFileDialog.getOpenFileName(self,
                                                         'Open Recorded Audio',
                                                         'C:\\Users\\ASUS\\AppData\\Local\\Programs\\Python\\Python36\\BTP',
                                                         'Audio files (*.wav *.mp3)')
                self.nameOfAudio.setText(self.fname[0])
                self.loadAudio.setEnabled(True)
                
                
        def load(self):
            fs1 = self.str2int_fs()
            (self.wavFile,self.rate) = librosa.load(self.fname[0],sr=int(fs1),mono=True)
            time_duration = self.wavFile.size/self.rate
            pr = str(time_duration) + " Sec"
            self.timeDuration.setText(pr)
            self.plotEdit.setEnabled(True)
            self.importModel.setEnabled(True)
            self.Show.setEnabled(True)
            self.reset.setEnabled(True)
            self.cancelEdit.setEnabled(True)
       
        def Identify(self):
            a = ['AC & FAN','CRYING','MUSIC','SPEECH']
            return (a[self.mode[self.Index]])
        
        def Identify_wav(self):
            a = ['AC & FAN.wav','CRYING.wav','MUSIC.wav','SPEECH.wav']
            return (a[self.mode[self.Index]])
            
        def left(self):
            self.front.setEnabled(True)
            self.Index -= 1
            if(self.Index<=0):
                self.back.setEnabled(False)
            
            self.frameplot()
            start = "<< "+"{:.1f}".format(self.Index*self.time)+' sec.'
            self.startTime.setText(start)
            end = "{:.1f}".format((self.Index+1)*self.time)+' sec. >>'
            self.endTime.setText(end)
            show = self.Identify()
            show_wav = self.Identify_wav()
            p = "Frame " + str(self.Index+1) + " || " + show
            self.showFrame.setText(p)
            pygame.mixer.music.load(show_wav)
            pygame.mixer.music.play()
                
        def right(self):
            self.back.setEnabled(True)
            self.Index += 1
            if (self.Index>=self.mode.size-1):
                self.front.setEnabled(False)
            
            self.frameplot()
            start = "<< "+"{:.1f}".format(self.Index*self.time)+' sec.'
            self.startTime.setText(start)
            end = "{:.1f}".format((self.Index+1)*self.time)+' sec. >>'
            self.endTime.setText(end)
            show = self.Identify()
            show_wav = self.Identify_wav()
            p = "Frame " + str(self.Index+1) + " || " + show
            self.showFrame.setText(p)
            pygame.mixer.music.load(show_wav)
            pygame.mixer.music.play()
            
        def plot(self):
            self.figure.clear()
            ax = self.figure.add_subplot(111)
            x = np.arange(1,self.wavFile.size+1)
            x = np.divide(x,self.rate)
            ax.plot(x,self.wavFile,'b-')
            #ax.set_title('Uploaded Audio')
            self.canvas.draw()
            self.playFrame.setEnabled(True)
            self.passWavFile = self.wavFile
               
        def frameplot(self):   
            self.playFrame.setEnabled(True)
            self.figure.clear()
            start = int(self.Index*self.time*self.rate)
            end = int((self.Index+1)*self.time*self.rate)-1
            
            wave = self.wavFile[start:end]
            x = np.arange(1,wave.size+1)
            x = np.divide(x,self.rate)
            x = np.add(x,self.Index*self.time)
            ax = self.figure.add_subplot(111)
            ax.plot(x,wave,'b-')
            ax.set_title('Frame Number '+str(self.Index+1))
            self.canvas.draw()
            self.passWavFile = wave
            
        def playframe(self):
            sd.play(self.passWavFile,self.rate)
            
        
        """ short time fourier transform of audio signal """
        def stft(self,sig, frameSize, overlapFac=0.9, window=np.hanning):
                win = window(frameSize)
                hopSize = int(frameSize - np.floor(overlapFac * frameSize))

                # zeros at beginning (thus center of 1st window should be for sample nr. 0)   
                samples = np.append(np.zeros(int(np.floor(frameSize/2.0))), sig)    
                # cols for windowing
                cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
                # zeros at end (thus samples can be fully covered by frames)
                samples = np.append(samples, np.zeros(frameSize))

                frames = stride_tricks.as_strided(samples, shape=(int(cols), frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
                frames *= win

                return np.fft.rfft(frames)
            
        def logscale_spec(self,spec, sr=44100, factor=20.):
               
                timebins, freqbins = np.shape(spec)

                scale = np.linspace(0, 1, freqbins) ** factor
                scale *= (freqbins-1)/max(scale)
                scale = np.unique(np.round(scale))

                # create spectrogram with new freq bins
                newspec = np.complex128(np.zeros([timebins, len(scale)]))
                for i in range(0, len(scale)):        
                    if i == len(scale)-1:
                        newspec[:,i] = np.sum(spec[:,int(scale[i]):], axis=1)
                    else:        
                        newspec[:,i] = np.sum(spec[:,int(scale[i]):int(scale[i+1])], axis=1)

                # list center freq of bins
                allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
                freqs = []
                for i in range(0, len(scale)):
                    if i == len(scale)-1:
                        freqs += [np.mean(allfreqs[int(scale[i]):])]
                    else:
                        freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

                return newspec, freqs
        
        def plotstft(self, samples, samplerate, binsize=400, plotpath=None, colormap="jet"):
               
                plt.close('all')

                s = self.stft(samples, binsize)
                sshow, freq = self.logscale_spec(s, factor=1.0, sr=samplerate)
                ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel
                w = ims.shape[0]/1250.0
                h = ims.shape[1]/1250.0
                plt.figure(figsize=(w,h))
                plt.axis('off')
                fig = plt.imshow(ims.T, origin="lower", aspect="auto", cmap=colormap, interpolation="nearest")
                fig.axes.get_xaxis().set_visible(False)
                fig.axes.get_yaxis().set_visible(False)

                if plotpath:
                    plt.savefig(plotpath, bbox_inches="tight", pad_inches=0., dpi=1000)
                else:
                    plt.show()

                plt.clf()

                return ims  
        
        def importmodel(self):
            
                if (self.modelSelect.currentText()=='FNN'):
                    if (self.frameSizeEdit.currentText()=='100'):
                        self.my_model=load_model('manikanta_FeedforwardNN_4class_100ms.h5')
                        self.modelIndex = 0
                    elif (self.frameSizeEdit.currentText()=='250'):
                        self.my_model=load_model('manikanta_FeedforwardNN_4class_250ms.h5')
                        self.modelIndex = 0
                    elif (self.frameSizeEdit.currentText()=='500'):
                        self.my_model=load_model('manikanta_FeedforwardNN_4class_500ms.h5')
                        self.modelIndex = 0
                elif (self.modelSelect.currentText()=='1D CNN'):
                    if (self.frameSizeEdit.currentText()=='100'):
                        self.my_model=load_model('my_CNN_100ms_mani.h5')
                        self.modelIndex = 1
                    elif (self.frameSizeEdit.currentText()=='250'):
                        self.my_model=load_model('my_CNN_250ms_mani.h5')
                        self.modelIndex = 1
                    else :
                        self.my_model=load_model('my_CNN_500ms_mani.h5')
                        self.modelIndex = 1
                elif(self.modelSelect.currentText()=='SVM'):
                    if (self.frameSizeEdit.currentText()=='100'):
                        self.filename = 'SVM_100ms_Rbf_model.save'
                        self.scaler_file = "my_scaler_100ms.save"
                        self.modelIndex = 2
                    elif (self.frameSizeEdit.currentText()=='250'):
                        self.filename = 'SVM_250ms_Rbf_model.save'
                        self.scaler_file = "my_scaler_250ms.save"
                        self.modelIndex = 2
                    else :
                        self.filename = 'SVM_500ms_Rbf_model.save'
                        self.scaler_file = "my_scaler_500ms.save"
                        self.modelIndex = 2
                else:
                    if (self.frameSizeEdit.currentText()=='100'):
                        self.my_model=load_model('mani_spectrogrammodel_cnn_100ms.h5')
                        self.modelIndex = 1
                    elif (self.frameSizeEdit.currentText()=='250'):
                        self.my_model=load_model('mani_spectrogrammodel_cnn_250ms.h5')
                        self.modelIndex = 1
                    else :
                        self.my_model=load_model('mani_spectrogrammodel_cnn_500ms.h5')
                        self.modelIndex = 1
                
                self.processStart.setEnabled(True)                    
                print(self.modelIndex)
            
        def process(self):
                self.frameSize = self.str2int_framesize()
                if(self.modelIndex!=3):            
                    
                    print(self.frameSize)
                    self.overLap = self.frameSize/2
                    print(self.overLap)
                    print(self.rate*self.frameSize/1000)
                    print(math.log2(self.rate*self.frameSize/1000))
                    print(math.ceil(math.log2(self.rate*self.frameSize/1000)))
                    self.nfft = 2**(math.ceil(math.log2(self.rate*self.frameSize/1000)))
                    self.mfcc = python_speech_features.base.mfcc(self.wavFile, samplerate=self.rate, winlen=self.frameSize/1000, winstep=self.overLap/1000, numcep=13, nfilt=26,
                            nfft=self.nfft, lowfreq=0, highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=True)
                    self.csvData = self.mfcc

                    with open('prediction.csv','w') as csvFile:
                        writer = csv.writer(csvFile)
                        writer.writerows(self.csvData)

                    csvFile.close()
                else:
                    self.classes = ['Spectogram']
                    fs = self.rate
                    fsize = int(fs/2)
                    hop = fsize // 10
                    hop_length = fsize // 40
                    num_classes = len(self.classes)
                    y = self.wavFile
                    rng = y.shape[0]//hop - 1
                    for i in range(0, rng):
                        data = y[i*hop:i*hop+fsize]
                        path = './Test_2D_CNN/Spectogram/' + str(i+1) + '.png'
                        ims = self.plotstft(data, fs, plotpath=path)
                    self.img_height,self.img_width = 128,128
                    self.test_datagen = ImageDataGenerator(rescale=1./255)
                    self.test_generator = self.test_datagen.flow_from_directory('./Test_2D_CNN',target_size=(self.img_height,self.img_width),
                                                  batch_size=21,
                                                  shuffle=False,
                                                  classes=self.classes,
                                                  class_mode='categorical')
                    
                self.predictStart.setEnabled(True)
                
       
        def predict(self):
            
                
            
                if(self.modelIndex==2):
                    df1=pd.read_csv("prediction.csv",na_values=['NA','?'])
                    df1.columns=['MFCC0', 'MFCC1','MFCC2','MFCC3','MFCC4','MFCC5','MFCC6','MFCC7','MFCC8', 
                    'MFCC9', 'MFCC10' ,'MFCC11', 'MFCC12']
                    loaded_model_rbf1 = joblib.load(open(self.filename,'rb'))
                    scaler = joblib.load(self.scaler_file) 
                    X = scaler.transform(df1) 

                    holdout_pred_rbf1 = loaded_model_rbf1.predict(X)
                    self.test_pred1 = np.empty(holdout_pred_rbf1.size,dtype='int')
                    c = 0
                    for i in holdout_pred_rbf1:
                        if(i=='AC & FAN'):
                            self.test_pred1[c] = 0
                        elif(i=='CRYING'):
                            self.test_pred1[c] = 1
                        elif(i=='MUSIC'):
                            self.test_pred1[c] = 2
                        else:
                             self.test_pred1[c] = 3
                        c += 1
                             
                    print(holdout_pred_rbf1)
                    print(self.test_pred1)
                elif(self.modelIndex==3):
                    self.test_generator.reset()
                    self.test_steps = self.test_generator.n//self.test_generator.batch_size
                    self.y_test = self.my_model.predict_generator(self.test_generator,
                                                            steps=self.test_steps,
                                                            verbose=1)
                    self.test_pred1 = np.argmax(self.y_test, axis=1).astype('int8')
                    print(self.test_pred1)
                    filelist = glob.glob(os.path.join('./Test_2D_CNN/Spectogram/', "*.png"))
                    for f in filelist:
                        os.remove(f)
                else:
                    df1=pd.read_csv("prediction.csv")
                    df1.columns=['MFCC0', 'MFCC1','MFCC2','MFCC3','MFCC4','MFCC5','MFCC6','MFCC7','MFCC8', 'MFCC9', 'MFCC10' ,'MFCC11', 'MFCC12']
                    if (self.modelIndex==1):
                        df1 = np.expand_dims(df1,axis=2)
                    my_his1=self.my_model.predict(df1)
                    self.test_pred1 = np.argmax(my_his1, axis=1)
                    print(self.test_pred1)
                    
                    
                self.size = self.test_pred1.size
                self.winLen = self.str2int_winlen()
                self.predictionFrame = int(2*(self.winLen/self.frameSize))
                NPadding = (-self.test_pred1.size)%self.predictionFrame
                random = -np.random.randint(1,20000000,size=NPadding)
                self.test_pred1 = np.concatenate((self.test_pred1,random),axis=0)
                #self.test_pred1 = np.pad(self.test_pred1, (0,NPadding), 'constant', constant_values=(0,-1))
                self.test_pred2 = librosa.util.frame(self.test_pred1, frame_length=self.predictionFrame, hop_length=self.predictionFrame)
                self.test_pred2 = self.test_pred2.T
                self.mode = stats.mode(self.test_pred2, axis=1)
                self.mode = self.mode[0][:, 0]
                print(self.mode)
            
                self.time = self.winLen/1000
                if(self.modelIndex!=3):
                    os.remove('prediction.csv')
                self.predictionRecord.setEnabled(True)
            
        def record(self):
            self.Index = 0
            self.front.setEnabled(True)
            c_acfan=sum(self.test_pred1==0)
            c_crying=sum(self.test_pred1==1)
            c_music=sum(self.test_pred1==2)
            c_speech=sum(self.test_pred1==3)
            
            
            
            print('AC & FAN Predicted : ',100*c_acfan/self.size)
            print('CRYING Predicted : ',100*c_crying/self.size)
            print('MUSIC Predicted : ',100*c_music/self.size)
            print('SPEECH Predicted : ',100*c_speech/self.size)
            
            
               
            pr = 'AF: '+"{:.2f}".format(100*c_acfan/self.size) + '||' + 'C: ' + "{:.2f}".format(100*c_crying/self.size) + '||' + 'M: '+"{:.2f}".format(100*c_music/self.size)+ '||' + 'S: '+"{:.2f}".format(100*c_speech/self.size)

            self.totalPredictionResult.setText(pr)
            self.frameplot()
            show = self.Identify()
            show_wav = self.Identify_wav()
            p = "Frame "+str(self.Index+1) + " || " + show
            self.startTime.setText('<< 0 sec')
            self.showFrame.setText(p)
            self.endTime.setText(str(self.time)+' sec >>')
            pygame.mixer.music.load(show_wav)
            pygame.mixer.music.play()
            
            
        def modelShow(self):
            img_name = self.modelGraph.currentText()
            frameS = self.frameSizeEdit.currentText()
            modelN = self.modelSelect.currentText()
            image_name = frameS+'_'
            if(modelN=='FNN'):
                image_name += 'FNN_'
            elif(modelN=='1D CNN'):
                image_name += 'CNN_'
            else:
                image_name += 'SVM_'
            if(img_name=='K-fold Accuracy'):
                image_name += 'acc'
            elif(img_name=='K-fold Loss'):
                image_name += 'loss'
            else:
                image_name += 'cm'
        
            pixmap = QtGui.QPixmap(image_name+'.png')
            self.lbl.setPixmap(pixmap)

        def cancel(self):
                print('Cancelled')
                self.close()
        
        def Reset(self):
            self.figure.clear()
            self.loadAudio.setEnabled(False)
            self.plotEdit.setEnabled(False)
            self.playFrame.setEnabled(False)
            self.processStart.setEnabled(False)
            self.Show.setEnabled(False)
            self.back.setEnabled(False)
            self.front.setEnabled(False)
            self.predictionRecord.setEnabled(False)
            self.predictStart.setEnabled(False)
            self.Index = 0
            self.figure.clear()
            self.nameOfAudio.setText(' ')
            self.timeDuration.setText('')
            self.totalPredictionResult.setText('')
            self.showFrame.setText('')
            self.startTime.setText('')
            self.endTime.setText('')
            self.lbl.clear()
            self.reset.setEnabled(False)

In [ ]:
if __name__ == '__main__':
##
        app = QApplication(sys.argv) #Every PyQt5class Window(QWidget): #inherits from the QWidget class. QWidget widget is the base class of all user interface objects in PyQt5.
        window = Window()
        sys.exit(app.exec_()) #enters the mainloop of the application. The event handling starts from this point.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0
100
50.0
1600.0
10.643856189774725
11
[3 3 3 ... 3 3 3]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3]
AC & FAN Predicted :  3.8365304420350292
CRYING Predicted :  0.041701417848206836
MUSIC Predicted :  0.0
SPEECH Predicted :  96.12176814011677


error: Couldn't open 'SPEECH.wav'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
1
100
50.0
1600.0
10.643856189774725
11
[3 3 0 ... 3 0 0]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 0 3 0 3 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 0 0 3 3 0 3 3 3 3 3 3 3 3 3 0 0 3 3 3 3 3 0 3 3 3 3
 3 3 3 3 3 3 3 3 0]
AC & FAN Predicted :  30.73394495412844
CRYING Predicted :  0.0
MUSIC Predicted :  0.0
SPEECH Predicted :  69.26605504587155


error: Couldn't open 'SPEECH.wav'

2
100
50.0
1600.0
10.643856189774725
11


...

1
500
250.0
8000.0
12.965784284662087
13
[0 3 3 0 0 0 0 3 3 3 0 0 3 3 3 3 0 0 3 0 0 0 0 0 0 3 0 0 1 0 0 0 3 0 0 0 0
 3 3 3 0 3 0 0 0 3 1 0 3 3 3 0 0 0 0 3 0 1 0 0 0 0 3 3 0 0 3 0 3 0 0 0 0 3
 0 3 3 0 0 3 3 3 3 3 0 3 3 3 3 0 0 0 0 3 3 3 3 3 3 0 0 0 3 3 0 3 3 0 3 3 3
 3 0 0 3 0 0 0 3 3 3 2 0 0 3 0 0 0 3 1 0 3 3 0 0 3 0 0 0 0 3 3 0 0 3 3 0 0
 0 3 3 3 3 3 3 3 3 3 3 0 0 0 0 3 1 0 3 3 0 0 0 3 0 0 0 0 3 3 0 3 3 0 3 3 0
 3 3 0 0 0 2 3 3 3 3 3 3 0 0 0 0 0 0 0 0 3 3 3 3 0 0 0 3 0 0 0 3 3 3 3 0 0
 0 0 0 0 0 3 0 0 0 3 0 3 3 0 0 0 0 0 3 3 3 3 3 3 3 0 0 0 0 0 0 3 0 0 3 3 3
 3 0 3 0 0 0 3 3 3 3 0 0 0 0 3 3 3 3 0 0 3 3 0 0 3 3 3 3 3 1 0 0 0 0 3 3 0
 3 3 0 1 3 0 3 3 0 0 3 3 3 3 0 3 0 0 0 3 3 3 3 3 3 0 0 3 3 0 0 0 3 0 0 3 3
 3 3 3 3 0 0 2 3 0 0 0 0 0 0 3 0 0 0 0 0 3 0 0 0 0 3 3 3 3 3 0 0 3 3 3 3 3
 3 3 3 3 3 3 3 0 1 3 3 3 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 3 3 3 0 0
 3 3 3 0 0 3 0 0 3 0 0 3 0 0 0 0 0 3 0 0 0 3 3 0 0 0 3 0 3 3 0 3 3 3 3 3 3
 3 3 0 0 3 0 0 0 0 3 1 3 0 0 3 0 3 3 3 0 0 0 0 0 0 3 3 3 3 

error: Couldn't open 'AC & FAN.wav'

1
500
250.0
8000.0
12.965784284662087
13
[1 1 1 3 3 3 2 2 2 2 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 3 3 1 1 3 3 3 3
 1 1 1 1 1 1 1 3 3 3 3 1 1 3 2 2 2 2 1 1 1 1 1 1 3 3 3 1 1 3 1 1 3 3 3 3 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 2 2 2 1 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 3 3 3 3 3 3 3 1 1 3 3 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 2 2 2 3 1
 1 1 1 1 1 1 1 1 3 3 3 3 1 3 3 3 3 3 3 3 3 1 3 3 1 1 1 1 1 1 3 1 3 1 1 1 1
 1 1 3 2 2 2 3 3 3 3 3 1 1 1 3 1 1 1 3 1 3 3 3 3 3 1 1 3 3 3 3 3 3 3 1 1 1
 1 3 1 1 3 3 2 1 1 1 2 2 2 2 3 3 1 1 1 1 1 1 1 1 1 3 3 3 3 3 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 1
 3 3 3 3 3 3 1 1 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 2 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

error: Couldn't open 'CRYING.wav'

1
250
125.0
4000.0
11.965784284662087
12
